In [14]:
from ultralytics import YOLO
import cv2
import os
import json
from tqdm import tqdm

# ---------------- CONFIG ----------------
MODEL_PATH = r"runs\detect\train\weights\best.pt"
IMG_DIR = r"test\images"          # original images
ROI_DIR = r"roii"
META_DIR = r"roi_metaaaa"

CONF_THRES = 0.5
EXPAND_SCALE = 1.5
# ---------------------------------------

os.makedirs(ROI_DIR, exist_ok=True)
os.makedirs(META_DIR, exist_ok=True)

model = YOLO(MODEL_PATH)

image_files = [f for f in os.listdir(IMG_DIR)
               if f.lower().endswith((".jpg", ".png", ".jpeg"))]

for img_name in tqdm(image_files):
    img_path = os.path.join(IMG_DIR, img_name)
    img = cv2.imread(img_path)

    if img is None:
        continue

    H, W = img.shape[:2]

    results = model(img, conf=CONF_THRES, device=0)
    result = results[0]

    if len(result.boxes) == 0:
        continue  # no stenosis detected

    # ---- assume best detection ----
    box = result.boxes.xyxy[0].cpu().numpy()
    conf = float(result.boxes.conf[0])

    x1, y1, x2, y2 = map(int, box)

    # ---- expand ROI ----
    cx = (x1 + x2) // 2
    cy = (y1 + y2) // 2

    bw = int((x2 - x1) * EXPAND_SCALE)
    bh = int((y2 - y1) * EXPAND_SCALE)

    x1_p = max(0, cx - bw)
    y1_p = max(0, cy - bh)
    x2_p = min(W, cx + bw)
    y2_p = min(H, cy + bh)

    roi = img[y1_p:y2_p, x1_p:x2_p]

    if roi.size == 0:
        continue

    # ---- YOLO box in ROI coordinates ----
    sx1 = x1 - x1_p
    sy1 = y1 - y1_p
    sx2 = x2 - x1_p
    sy2 = y2 - y1_p

    roi_h, roi_w = roi.shape[:2]

    # clamp
    sx1 = max(0, min(sx1, roi_w - 1))
    sx2 = max(0, min(sx2, roi_w - 1))
    sy1 = max(0, min(sy1, roi_h - 1))
    sy2 = max(0, min(sy2, roi_h - 1))

    base = os.path.splitext(img_name)[0]

    # ---- save ROI ----
    roi_path = os.path.join(ROI_DIR, img_name)
    cv2.imwrite(roi_path, roi)

    # ---- save metadata ----
    meta = {
        "yolo_box": [int(sx1), int(sy1), int(sx2), int(sy2)],
        "confidence": conf
    }

    meta_path = os.path.join(META_DIR, base + ".json")
    with open(meta_path, "w") as f:
        json.dump(meta, f, indent=2)

print("✅ Batch YOLO metadata generation completed")


  0%|          | 0/1249 [00:00<?, ?it/s]


0: 640x640 1 Stenosis, 4.5ms
Speed: 3.7ms preprocess, 4.5ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)


  0%|          | 1/1249 [00:00<02:08,  9.73it/s]


0: 640x640 1 Stenosis, 5.0ms
Speed: 2.9ms preprocess, 5.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Stenosis, 6.0ms
Speed: 2.6ms preprocess, 6.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Stenosis, 11.3ms
Speed: 3.9ms preprocess, 11.3ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Stenosis, 8.1ms
Speed: 0.0ms preprocess, 8.1ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Stenosis, 3.9ms
Speed: 2.5ms preprocess, 3.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Stenosis, 8.8ms
Speed: 3.1ms preprocess, 8.8ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)


  1%|          | 7/1249 [00:00<00:32, 37.64it/s]

...


0: 640x640 1 Stenosis, 6.8ms
Speed: 2.8ms preprocess, 6.8ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Stenosis, 10.8ms
Speed: 2.5ms preprocess, 10.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Stenosis, 5.0ms
Speed: 2.0ms preprocess, 5.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Stenosis, 5.7ms
Speed: 2.8ms preprocess, 5.7ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 2.4ms
Speed: 3.0ms preprocess, 2.4ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Stenosis, 11.0ms
Speed: 4.0ms preprocess, 11.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


100%|█████████▉| 1246/1249 [00:23<00:00, 54.94it/s]


0: 640x640 1 Stenosis, 5.1ms
Speed: 4.0ms preprocess, 5.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Stenosis, 5.0ms
Speed: 3.0ms preprocess, 5.0ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Stenosis, 11.6ms
Speed: 3.0ms preprocess, 11.6ms inference, 3.5ms postprocess per image at shape (1, 3, 640, 640)


100%|██████████| 1249/1249 [00:23<00:00, 52.47it/s]

✅ Batch YOLO metadata generation completed


In [9]:
import json, cv2

roi = cv2.imread(r"roi\14_092_1_0067_bmp.rf.1f34545c344b76b2570c1c7a365e1809.jpg")

with open("roi_meta/14_092_1_0067_bmp.rf.1f34545c344b76b2570c1c7a365e1809.json") as f:
    meta = json.load(f)

sx1, sy1, sx2, sy2 = meta["yolo_box"]

debug = roi.copy()
cv2.rectangle(debug, (sx1, sy1), (sx2, sy2), (0,0,255), 2)

cv2.imshow("YOLO box in ROI", debug)
cv2.waitKey(0)


-1